评：这个方法的核心组成
1. Planner
2. Executor
3. Resolver

![img](./arch.jpeg)

`Planner`将任务拆分成一个个可以连续执行的子任务，每个子任务都是
>
> Plan: reasoning
>
> #E1 = Tool[argument for tool]
>
的格式。然后将Planner的输出转换成列表`Array<[plan, stepName, tool, toolInput]>`。

`Executor`一次执行上面的列表，同时维护一个results纪录。 `Executor`首先会从`results`中取出键值对，对`toolInput`进行文本替换。然后会将`toolInput`作为输入，执行`tool`，并将`tool`的执行结果放到`results`中。执行完所有的任务后，`Executor`会讲控制权交给`Resolver`，由`Resolver`总结出最终结果并输出。


由此可见，`ReWOO`的核心优势在于
1. 执行过程清晰，消耗少。一个任务需要执行多少步，在`Planner`阶段就能确定了
2. 需要的token少。在tool的执行阶段，只需要toolInput作为输入


对比`hierarchical_agent_teams`，`supervisor`可以类比`Planner`，各个`Team`可以类比`Executor`中的`tools`

In [ ]:
import './../../loadenv.mjs'

In [2]:
import { Annotation } from '@langchain/langgraph'

const GraphState = Annotation.Root({
    task: Annotation<string>({
        reducer: (x, y) => y ?? x,
        default: () => '',
    }),
    planString: Annotation<string>({
        reducer: (x, y) => y ?? x,
        default: () => '',
    }),
    steps: Annotation<string[][]>({
        reducer: (x, y) => x.concat(y),
        default: () => [],
    }),
    results: Annotation<Record<string, any>>({
        reducer: (x, y) => ({ ...x, ...y }),
        default: () => ({}),
    }),
    result: Annotation<string>({
        reducer: (x, y) => y ?? x,
        default: () => '',
    }),
})

# Planner

In [3]:
import { getModel } from './../../utils.mjs'

const model = getModel()

In [4]:
import { ChatPromptTemplate } from '@langchain/core/prompts'

const template = `For the following task, make plans that can solve the problem step by step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:
(1) Google[input]: Worker that searches results from Google. Useful when you need to find short and succinct answers about a specific topic. The input should be a search query.
(2) LLM[input]: A pre-trained LLM like yourself. Useful when you need to act with general world knowledge and common sense. Prioritize it when you are confident in solving the problem yourself. Input can be any instruction.

For example,
Task: Thomas, Toby, and Rebecca worked a total of 157 hours in one week. Thomas worked x hours. Toby worked 10 hours less than twice what Thomas worked, and Rebecca worked 8 hours less than Toby. How many hours did Rebecca work? 
Plan: Given Thomas worked x hours, translate the problem into algebraic expressions and solve with Wolfram Alpha.
#E1 = WolframAlpha[Solve x + (2x - 10) + ((2x - 10) - 8) = 157]
Plan: Find out the number of hours Thomas worked.
#E2 = LLM[What is x, given #E1]
Plan: Calculate the number of hours Rebecca worked.
#E3 = Calculator[(2 * #E2 - 10) - 8]

Important!
Variables/results MUST be referenced using the # symbol!
The plan will be executed as a program, so no coreference resolution apart from naive variable replacement is allowed.
The ONLY way for steps to share context is by including #E<step> within the arguments of the tool.

Begin! 
Describe your plans with rich details. Each Plan should be followed by only one #E.

Task: {task}`

const promptTemplate = ChatPromptTemplate.fromMessages([
    ['human', template],
])

const planner = promptTemplate.pipe(model)

In [5]:
// const task = 'what is the hometown of the winner of the 2023 australian open?'
// await planner.invoke({ task })

## Planner Node

In [6]:
import { RunnableConfig } from '@langchain/core/runnables'

const regexPattern = new RegExp(
    "Plan\\s*\\d*:\\s*([^#]+)\\s*(#E\\d+)\\s*=\\s*(\\w+)\\s*\\[([^\\]]+)\\]",
    "g",
)

async function getPlan(state: typeof GraphState.State, config?: RunnableConfig) {
    console.log('---GET PLAN---')
    const task = state.task
    const result = await planner.invoke({ task }, config)
    const matches = result.content.toString().matchAll(regexPattern)
    let steps: string[][] = []
    for (const match of matches) {
        const item = [match[1], match[2], match[3], match[4], match[0]]
        if (item.some(i => i === undefined)) {
            throw new Error('Invalid match')
        }
        steps.push(item as string[])
    }
    return {
        steps,
        planString: result.content.toString(),
    }
}

# Executor

In [7]:
import { TavilySearchResults } from '@langchain/community/tools/tavily_search'

const search = new TavilySearchResults()

In [8]:
const _getCurrentTask = (state: typeof GraphState.State) => {
    console.log('_getCurrentTask, ', state)
    if (!state.results) {
        return 1
    }
    if (Object.entries(state.results).length === state.steps.length) {
        return null
    }
    return Object.entries(state.results).length + 1
}

const _parseResult = (input: unknown) => {
    if (typeof input === 'string') {
        const parsedInput = JSON.parse(input)
        if (Array.isArray(parsedInput) && 'content' in parsedInput[0]) {
            return parsedInput.map(({ content }) => content).join('\n')
        }
    }
    if (input && typeof input === 'object' && 'content' in input) {
        return input.content
    }
    throw new Error('Invalid input received')
}

async function toolExecution(state: typeof GraphState.State, config?: RunnableConfig) {
    console.log('---EXECUTE TOOL---')
    const _step = _getCurrentTask(state)
    if (_step === null) {
        throw new Error('No current task found')
    }
    const [_, stepName, tool, toolInputTemplate] = state.steps[_step - 1]
    let toolInput = toolInputTemplate
    const _results = state.results || {}
    for (const [k, v] of Object.entries(_results)) {
        toolInput = toolInput.replace(k, v)
    }
    let result
    if (tool === 'Google') {
        result = await search.invoke(toolInput, config)
    } else if (tool === 'LLM') {
        result = await model.invoke(toolInput, config)
    } else {
        throw new Error('Invalid tool specified')
    }
    _results[stepName] = JSON.stringify(_parseResult(result), null, 2)
    return { results: _results }
}

# Solver 

In [9]:
const solvePrompt = ChatPromptTemplate.fromTemplate(`Solve the following task or problem. To solve the problem, we have made step-by-step Plan and retrieved corresponding Evidence to each Plan. Use them with caution since long evidence might contain irrelevant information.

{plan}

Now solve the question or task according to provided Evidence above. Respond with the answer directly with no extra words.

Task: {task}
Response:`
)

async function solve(state: typeof GraphState.State, config?: RunnableConfig) {
    console.log('---SOLVE---')
    let plan = ''
    const _results = state.results || {}
    for (let [_plan, stepName, tool, toolInput] of state.steps) {
        for (const [k, v] of Object.entries(_results)) {
            toolInput = toolInput.replace(k, v)
        }
        plan += `Plan: ${_plan}\n${stepName} = ${tool}[${toolInput}]\n`
    }
    const model = getModel({ temperature: 0 })
    const result = await solvePrompt
        .pipe(model)
        .invoke({ plan, task: state.task }, config)
    return {
        result: result.content.toString(),
    }
}


# Define Graph

In [10]:
import { END, START, StateGraph } from '@langchain/langgraph'
import { MemorySaver } from '@langchain/langgraph'

const _route = (state: typeof GraphState.State) => {
    console.log('---ROUTE TASK---')
    const _step = _getCurrentTask(state)
    if (_step === null) {
        return 'solve'
    }
    return 'tool'
}

const workflow = new StateGraph(GraphState)
    .addNode('plan', getPlan)
    .addNode('tool', toolExecution)
    .addNode('solve', solve)
    .addEdge('plan', 'tool')
    .addEdge('solve', END)
    .addConditionalEdges('tool', _route)
    .addEdge(START, 'plan')

const app = workflow.compile({ checkpointer: new MemorySaver() })

In [ ]:
import { printGraph } from './../../utils.mjs'
await printGraph(app.getGraph())

In [ ]:
const threadConfig = { configurable: { thread_id: '123' }}
let finalResult
const stream = await app.stream(
    {
        task: 'what is the hometown of the winner of then 2023 australian open?',
    },
    threadConfig,
)
for await (const item of stream) {
    console.log(item)
    console.log('-----')
    finalResult = item
}

In [ ]:
const snapshot = await app.getState(threadConfig)
console.log(snapshot.values.result)